In [1]:
import os
import random
import torch
from tqdm import trange
import time
import numpy as np
import pickle
import json
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from biencoder import *
import data_process as data
from transformers import get_linear_schedule_with_warmup
from transformers import  BertTokenizerFast, BertModel

In [7]:
embeddings_folder = "entity_embdes_2.pkl"
input_batch_size = 64
model_path = 'C:\\Users\\aydxng\\Documents\\ds-fundingbodies-linkingcomponent-masterthesis\\Thesis\\LinkerRound1\\cand_model_2_epoch_0.pt'
seed = 0
device='cuda'

In [3]:
with open('entities_256.pkl','rb') as f:
    entity_dict=pickle.load(f)

In [4]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [5]:
from data_process import *
def process_entity_data(entity_dict):
    
    iter_ = list(entity_dict.keys())
    processed_samples = []
    for idx, sample in enumerate(iter_):
        
       
        entity_id = int(sample)
        entity_inputs = entity_dict[sample]['ids']
        
        record = {
                "entity_id": entity_id,
                "entity_inputs": entity_inputs
        }
        
        processed_samples.append(record)
    
    cand_vecs = torch.tensor(
        select_field(processed_samples, "entity_inputs"), dtype=torch.long,
    )
        
    label_idx = torch.tensor(
        select_field(processed_samples, "entity_id"), dtype=torch.long,
    )
    data = {
        "entity_id": label_idx,
        "entity_inputs": cand_vecs
    }
    tensor_data = TensorDataset(cand_vecs, label_idx)
    return data, tensor_data

In [8]:
cand_model = torch.load(model_path).to(device)

In [10]:
data, tensor_data = process_entity_data(entity_dict)

sampler = RandomSampler(tensor_data)
dataloader = DataLoader(tensor_data, sampler=sampler, batch_size=input_batch_size)

In [11]:
cand_model.eval()
entity_emebeddings = dict()
print(len(dataloader))
with torch.no_grad():
    start = time.time()
    for step, batch in enumerate(dataloader):
        if step%10==0:
            print("Step: ",step," ",time.time()-start)
        candidate_input, e_ids  = batch
        this_batch = e_ids.size(0)
        e_ids = e_ids.cpu().detach().numpy()
        cand_rep = cand_model(candidate_input.to(device))[0][:,0,:]
        for i in range(this_batch):
            entity_emebeddings[e_ids[i]] = cand_rep[i].cpu().detach().numpy()

421
Step:  0   0.014000415802001953
Step:  10   22.838997840881348
Step:  20   44.593995332717896
Step:  30   66.3539936542511
Step:  40   94.05799269676208
Step:  50   122.9549868106842
Step:  60   151.76598811149597
Step:  70   180.43398904800415
Step:  80   208.9899878501892
Step:  90   237.79398846626282
Step:  100   266.68120312690735
Step:  110   295.61732029914856
Step:  120   324.736430644989
Step:  130   353.64254879951477
Step:  140   382.50466871261597
Step:  150   411.2607910633087
Step:  160   440.08991169929504
Step:  170   468.6790382862091
Step:  180   497.44216299057007
Step:  190   526.1602871417999
Step:  200   554.815835237503
Step:  210   583.3498296737671
Step:  220   611.7298247814178
Step:  230   640.1718184947968
Step:  240   668.8478128910065
Step:  250   697.7228064537048
Step:  260   726.5558013916016
Step:  270   755.2607960700989
Step:  280   783.9387903213501
Step:  290   812.7757840156555
Step:  300   841.6807782649994
Step:  310   870.5247728824615
Step

In [12]:
with open(embeddings_folder,"wb") as f:
    pickle.dump(entity_emebeddings,f)